## Secret/Public Keys

In [1]:
original_sk = PrivateKey.from_bytes(bytes.fromhex("31d87cd97f07d1b44064254617adc76a54d4fc176a557e7d8760b0a1c7f050f9"))
original_pk = original_sk.get_g1()
print(f" original sk: {bytes(original_sk).hex()}")
print(f" original pk: {original_pk}")

 original sk: 31d87cd97f07d1b44064254617adc76a54d4fc176a557e7d8760b0a1c7f050f9
 original pk: b87af552cf4b079371b7c38f69f15f11f96e913c5a6c7ef09eb82cc0693e86a90e5c98fff952d2b4066be3d9292a8a84


## Synthentic keys
A new set of keys derived from the **original public key** and the hidden puzzle hash. There are used by the [standard transaction](./standard-txn.ipynb).

1. Get the synthetic offset ([calculate_synthetic_offset](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/p2_delegated_puzzle_or_hidden_puzzle.py#L86))
```
synthetic_offset = sha256(original_pk + hidden_puzzle_hash)
```

2. Get the synthetic offset public key
```synthetic_offset_sk = PrivateKey.from_bytes(synthetic_offset.to_bytes(32, "big"))
synthetic_offset_pk = synthetic_offset_sk.get_g1()
```

3. Get the synthetic public key
```
synthentic_pk = original_pk + synthetic_offset_pk
```

In [3]:
# 1. Get the synthetic offset
synthetic_offset = calculate_synthetic_offset(original_pk, DEFAULT_HIDDEN_PUZZLE_HASH)
synthetic_offset_sk = PrivateKey.from_bytes(synthetic_offset.to_bytes(32, "big"))
print(f"synthetic_offset_sk:\t{bytes(synthetic_offset_sk).hex()}")

# 2. Get the synthetic offset public key
synthetic_offset_pk = synthetic_offset_sk.get_g1()
print(f"synthetic_offset_pk:\t{synthetic_offset_pk}")

# 3. Get the synthetic public key
synthetic_pk = original_pk + synthetic_offset_pk
print(f"synthetic_pk:\t\t{synthetic_pk}")

calculated_synthetic_pk = calculate_synthetic_public_key(original_pk, DEFAULT_HIDDEN_PUZZLE_HASH)
assert synthetic_pk == calculated_synthetic_pk

synthetic_offset_sk:	6d09189e26a65fd35f4ba150958489f696b85fc87cb7bb665b65f61e6fb23a80
synthetic_offset_pk:	b8aeaa286ce9d6fb34eac9228f6effc6b8692e470ea4316489bfce016fab3dd3f6954352c444b553254dbeb40fc25bf5
synthetic_pk:		8410a8a862f025d72b90ec667cd2a47dd1f8fe8667b14ff2cd90b62c104ba27acea8785ef3032db616eadad00069d298


### `synthentic_public_key = original_public_key + synthetic_offset_pubkey`

> When you add two public keys, the sum of their private keys gives the private key for the resulting public key.

In [9]:
calcualted_synthetic_sk = calculate_synthetic_secret_key(original_sk, DEFAULT_HIDDEN_PUZZLE_HASH)

# sum of two private keys
synthetic_sk = PrivateKey.aggregate([original_sk, synthetic_offset_sk])
# sum of two public keys
synthetic_pk = original_pk + synthetic_offset_pk 

print(f"synthetic_sk: {bytes(synthetic_sk).hex()}")
print(f"synthetic_pk: {synthetic_pk}")

assert synthetic_sk == calcualted_synthetic_sk
assert synthetic_pk == synthetic_sk.get_g1()
assert original_pk + synthetic_offset_pk == synthetic_sk.get_g1()

synthetic_sk: 2af3ee247c10b43f6c75ee8ea390795b97cfb7dce70edde4e2c6a6c137a28b78
synthetic_pk: 8410a8a862f025d72b90ec667cd2a47dd1f8fe8667b14ff2cd90b62c104ba27acea8785ef3032db616eadad00069d298


## Signing Message and Verifying Signature

In [10]:
message = std_hash(bytes("hello", "utf-8"))

signature = AugSchemeMPL.sign(synthetic_sk, message)

assert AugSchemeMPL.verify(synthetic_pk, message, signature)